Load dataset

In [1]:
import pandas as pd
import csv

In [2]:
class DataSet():
    def __init__(self, name="train", path="dataset"):
        self.path = path

        print("Reading dataset in %s/" %path)
        bodies = name+"_bodies.csv"
        stances = name+"_stances.csv"

        print("Loading files %s, %s" %(bodies,stances))
        self.data = self.read(bodies,stances)

    def read(self,bodies,stances):
        train_bodies = pd.read_csv(self.path + "/" + bodies)
        train_stances = pd.read_csv(self.path + "/" + stances)
        merged = train_bodies.merge(train_stances,left_on='Body ID',right_on='Body ID',how='outer')
        return merged
    
    def print(self):
        print(self.data.head())

In [3]:
training = DataSet()
test = DataSet(name="competition_test")
#training.data.to_pickle("training_data.pkl")

Reading dataset in dataset/
Loading files train_bodies.csv, train_stances.csv
Reading dataset in dataset/
Loading files competition_test_bodies.csv, competition_test_stances.csv


In [4]:
import os
import re
import nltk
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm

_wnl = nltk.WordNetLemmatizer() 

def normalize_word(w):
    return _wnl.lemmatize(w).lower()

def get_tokenized_lemmas(s):
    return [normalize_word(t) for t in nltk.word_tokenize(s)]

def clean(s):
    # Cleans a string: Lowercasing, trimming, removing non-alphanumeric
    return " ".join(re.findall(r'\w+', s, flags=re.UNICODE)).lower()

def remove_stopwords(l):
    # Removes stopwords from a list of tokens
    return [w for w in l if w not in feature_extraction.text.ENGLISH_STOP_WORDS]

def get_clean_tokens(s):
    s = clean(s)
    tokens = get_tokenized_lemmas(s)
    clean_tokens = remove_stopwords(tokens)
    return clean_tokens

In [5]:
training.data['articleBody tokens'] = training.data['articleBody'].map(get_clean_tokens)
training.data['Headline tokens'] = training.data['Headline'].map(get_clean_tokens)

In [172]:
from collections import Counter

#tunable parameter
min_len = 2

count_words = Counter()
for tokens in training.data['articleBody tokens'].values:
    for token in tokens:
        if len(token)>min_len:
            count_words[token]+=1


In [173]:
count_words.most_common(10)
# word cloud

[('said', 134112),
 ('state', 44137),
 ('video', 43653),
 ('year', 42425),
 ('report', 40928),
 ('apple', 40879),
 ('time', 37308),
 ('isi', 36373),
 ('people', 35930),
 ('told', 35424)]

Tokenize

In [7]:
vocabulary = training.data['articleBody tokens'].values + training.data['Headline tokens'].values 

In [8]:
listed_docs = list(training.data['articleBody'].values)
listed_docs2 = list(training.data['Headline'].values)

Vectorize - fit: Fit vocabulary means generate the vector space for the words

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(2,2), min_df=5)
vectorizer.fit(listed_docs)
vectorizer.fit(listed_docs2)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(2, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

Vectorize - transform: Represent a document on the vector space that was fit previously

In [11]:
training.data['tfidf bodies'] = vectorizer.transform(listed_docs)

In [13]:
training.data['tfidf headlines'] = vectorizer.transform(listed_docs2)

In [14]:
print(training.data.head())

   Body ID                                        articleBody  \
0        0  A small meteorite crashed into a wooded area i...   
1        0  A small meteorite crashed into a wooded area i...   
2        0  A small meteorite crashed into a wooded area i...   
3        0  A small meteorite crashed into a wooded area i...   
4        0  A small meteorite crashed into a wooded area i...   

                                            Headline     Stance  \
0  Soldier shot, Parliament locked down after gun...  unrelated   
1  Tourist dubbed ‘Spider Man’ after spider burro...  unrelated   
2  Luke Somers 'killed in failed rescue attempt i...  unrelated   
3   BREAKING: Soldier shot at War Memorial in Ottawa  unrelated   
4  Giant 8ft 9in catfish weighing 19 stone caught...  unrelated   

                                  articleBody tokens  \
0  [small, meteorite, crashed, wooded, area, nica...   
1  [small, meteorite, crashed, wooded, area, nica...   
2  [small, meteorite, crashed, wooded,

Save data just to make it less heavy

In [15]:
training.data.to_pickle('data_tfidf.pkl')

In [16]:
print(listed_docs[0])

A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports. 

Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. 
Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.

The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.

Humberto Garcia

In [17]:
a = vectorizer.transform([listed_docs[0]])
print(a)

  (0, 6540)	0.270362121695911
  (0, 6328)	0.40880455331186605
  (0, 4678)	0.20606023018732983
  (0, 4582)	0.23969388029179453
  (0, 4532)	0.24399338064741213
  (0, 3150)	0.25560080851563965
  (0, 2570)	0.2450334731861741
  (0, 1830)	0.2450334731861741
  (0, 1653)	0.47938776058358906
  (0, 1127)	0.21618092366193067
  (0, 310)	0.2798814335000886
  (0, 68)	0.2450334731861741


Measuring similarity between two documents

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
a = vectorizer.transform([listed_docs[0]])
b = vectorizer.transform([listed_docs[200]])
print(cosine_similarity(a,b))

[[0.]]


Find the headline closest to the first body

In [23]:
# find the headline closest to the first body:
body = vectorizer.transform([listed_docs[0]])
sim_new = 0
sim_old = 0
for indx, headline in enumerate(listed_docs2):
    head_vectorized = vectorizer.transform([headline])
    sim_new = cosine_similarity(body,head_vectorized)
    if sim_new > sim_old:
        sim_old = sim_new
        most_similar_indx = indx

In [24]:
print("Similarity: %f" %sim_old)
print("Most similar: %s" %listed_docs2[most_similar_indx])
print("Match for: %s" %listed_docs[0])

Similarity: 0.271787
Most similar: Meteor Leaves 40-Foot Crater Near Managua's Airport
Match for: A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports. 

Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. 
Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.

The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the commi

# Reintroduce the task

Ok, so let's do some fake news detection.

Given a headline and a body of text, we want to say whether these two are:

* unrelated
* agree with each other
* disagree with each other
* discuss each other


# Features ?

First, we have to generate features:

In [ ]:
def word_overlap_features(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        clean_headline = get_tokenized_lemmas(clean_headline)
        clean_body = get_tokenized_lemmas(clean_body)
        features = [
            len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))]
        X.append(features)
    return X

def gen_or_load_feats(feat_fn, headlines, bodies, feature_file):
    if not os.path.isfile(feature_file):
        feats = feat_fn(headlines, bodies)
        np.save(feature_file, feats)

    return np.load(feature_file)


def word_overlap_features(headlines, bodies):
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        clean_headline = get_tokenized_lemmas(clean_headline)
        clean_body = get_tokenized_lemmas(clean_body)
        features = [
            len(set(clean_headline).intersection(clean_body)) / float(len(set(clean_headline).union(clean_body)))]
        X.append(features)
    return X

def refuting_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        # 'refute',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_headline = get_tokenized_lemmas(clean_headline)
        features = [1 if word in clean_headline else 0 for word in _refuting_words] #how many refutting words in the headline
        X.append(features)
    return X


def polarity_features(headlines, bodies):
    _refuting_words = [
        'fake',
        'fraud',
        'hoax',
        'false',
        'deny', 'denies',
        'not',
        'despite',
        'nope',
        'doubt', 'doubts',
        'bogus',
        'debunk',
        'pranks',
        'retract'
    ]

    def calculate_polarity(text):
        tokens = get_tokenized_lemmas(text)
        return sum([t in _refuting_words for t in tokens]) % 2
    
    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        clean_headline = clean(headline)
        clean_body = clean(body)
        features = []
        features.append(calculate_polarity(clean_headline))
        features.append(calculate_polarity(clean_body))
        X.append(features)
    return np.array(X)


def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def chargrams(input, n):
    output = []
    for i in range(len(input) - n + 1):
        output.append(input[i:i + n])
    return output


def append_chargrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in chargrams(" ".join(remove_stopwords(text_headline.split())), size)]
    grams_hits = 0
    grams_early_hits = 0
    grams_first_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
        if gram in text_body[:100]:
            grams_first_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    features.append(grams_first_hits)
    return features

def append_ngrams(features, text_headline, text_body, size):
    grams = [' '.join(x) for x in ngrams(text_headline, size)]
    grams_hits = 0
    grams_early_hits = 0
    for gram in grams:
        if gram in text_body:
            grams_hits += 1
        if gram in text_body[:255]:
            grams_early_hits += 1
    features.append(grams_hits)
    features.append(grams_early_hits)
    return features

def hand_features(headlines, bodies):
    def binary_co_occurence(headline, body):
        # Count how many times a token in the title
        # appears in the body text.
        bin_count = 0
        bin_count_early = 0
        for headline_token in clean(headline).split(" "):
            if headline_token in clean(body):
                bin_count += 1
            if headline_token in clean(body)[:255]:
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def binary_co_occurence_stops(headline, body):
        # Count how many times a token in the title
        # appears in the body text. Stopwords in the title
        # are ignored.
        bin_count = 0
        bin_count_early = 0
        for headline_token in remove_stopwords(clean(headline).split(" ")):
            if headline_token in clean(body):
                bin_count += 1
                bin_count_early += 1
        return [bin_count, bin_count_early]

    def count_grams(headline, body):
        # Count how many times an n-gram of the title
        # appears in the entire body, and intro paragraph
        clean_body = clean(body)
        clean_headline = clean(headline)
        features = []
        features = append_chargrams(features, clean_headline, clean_body, 2)
        features = append_chargrams(features, clean_headline, clean_body, 8)
        features = append_chargrams(features, clean_headline, clean_body, 4)
        features = append_chargrams(features, clean_headline, clean_body, 16)
        features = append_ngrams(features, clean_headline, clean_body, 2)
        features = append_ngrams(features, clean_headline, clean_body, 3)
        features = append_ngrams(features, clean_headline, clean_body, 4)
        features = append_ngrams(features, clean_headline, clean_body, 5)
        features = append_ngrams(features, clean_headline, clean_body, 6)
        return features

    X = []
    for i, (headline, body) in tqdm(enumerate(zip(headlines, bodies))):
        X.append(binary_co_occurence(headline, body)
                 + binary_co_occurence_stops(headline, body)
                 + count_grams(headline, body))

    return X

In [ ]:

def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap."+name+".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting."+name+".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity."+name+".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "features/hand."+name+".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X,y


# Classification
Now, we choose our "favourite" classification algorithm

In [158]:
#check_version()
#parse_params()

#Load the training dataset and generate folds
d = DataSet()
folds,hold_out = kfold_split(d,n_folds=10)
fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)

# Load the competition dataset
competition_dataset = DataSet("competition_test")
X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

Xs = dict()
ys = dict()

# Load/Precompute all features now
X_holdout,y_holdout = generate_features(hold_out_stances,d,"holdout")
for fold in fold_stances:
    Xs[fold],ys[fold] = generate_features(fold_stances[fold],d,str(fold))


best_score = 0
best_fold = None


# Classifier for each fold
for fold in fold_stances:
    ids = list(range(len(folds)))
    del ids[fold]

    X_train = np.vstack(tuple([Xs[i] for i in ids]))
    y_train = np.hstack(tuple([ys[i] for i in ids]))

    X_test = Xs[fold]
    y_test = ys[fold]

    clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)
    clf.fit(X_train, y_train)

    predicted = [LABELS[int(a)] for a in clf.predict(X_test)]
    actual = [LABELS[int(a)] for a in y_test]

    fold_score, _ = score_submission(actual, predicted)
    max_fold_score, _ = score_submission(actual, actual)

    score = fold_score/max_fold_score

    print("Score for fold "+ str(fold) + " was - " + str(score))
    if score > best_score:
        best_score = score
        best_fold = clf


#Run on Holdout set and report the final score on the holdout set
predicted = [LABELS[int(a)] for a in best_fold.predict(X_holdout)]
actual = [LABELS[int(a)] for a in y_holdout]

print("Scores on the dev set")
report_score(actual,predicted)
print("")
print("")

#Run on competition dataset
predicted = [LABELS[int(a)] for a in best_fold.predict(X_competition)]
actual = [LABELS[int(a)] for a in y_competition]

print("Scores on the test set")
report_score(actual,predicted)

Reading dataset in dataset/
Loading files train_bodies.csv, train_stances.csv
   Body ID                                        articleBody  \
0        0  A small meteorite crashed into a wooded area i...   
1        0  A small meteorite crashed into a wooded area i...   
2        0  A small meteorite crashed into a wooded area i...   
3        0  A small meteorite crashed into a wooded area i...   
4        0  A small meteorite crashed into a wooded area i...   

                                            Headline     Stance  
0  Soldier shot, Parliament locked down after gun...  unrelated  
1  Tourist dubbed ‘Spider Man’ after spider burro...  unrelated  
2  Luke Somers 'killed in failed rescue attempt i...  unrelated  
3   BREAKING: Soldier shot at War Memorial in Ottawa  unrelated  
4  Giant 8ft 9in catfish weighing 19 stone caught...  unrelated  


NameError: name 'kfold_split' is not defined